# GCP Storage Tutorial

**Objective:** Provide sample code for uploading and downloading different file types.

**Author:** Phil Busko

In [1]:
import os, sys, json
import pandas as PD
import numpy as NP

In [2]:
MODULE_PATH = os.path.dirname(os.getcwd())
ROOT_PATH = os.path.dirname(MODULE_PATH)
print(MODULE_PATH)

C:\Projects\PraxiService\codebase\common


In [3]:
import io, base64
import cv2 as CV
import google.cloud.storage as GS
CREDENTIALS = os.path.join(ROOT_PATH, 'common', 'gcp-cred-datalake.json')
storageClient = GS.Client.from_service_account_json(CREDENTIALS)

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.display import clear_output

### Text / Json

In [5]:
# get the list of items

BUCKET = 'calpoly-2019'
items = list(storageClient.list_blobs(BUCKET, prefix='elitehrv'))
items = [t.name for t in items]

print( '\n'.join(items) )

elitehrv/all-athletes-2019.csv


In [6]:
# download file data for each item

storageBucket = storageClient.bucket(BUCKET)

for cloudPath in items:
    print(cloudPath)
    
    blob = storageBucket.blob(cloudPath)
    hrvBt = blob.download_as_string()
    
    stream = io.BytesIO(hrvBt)
    exampleDf = PD.read_csv(stream)

exampleDf.head()

elitehrv/all-athletes-2019.csv


,Member,Type,Date Time Start,Date Time End,Duration,HRV,Morning Readiness,Balance,HRV CV,HR,lnRmssd,Rmssd,Nn50,Pnn50,Sdnn,Low Frequency Power,High Frequency Power,LF/HF Ratio,Total Power
0,012065014,open,2019-09-23 06:54:45,NaN,53.417,77,NaN,NaN,0.0,57.68,4.97337,144.51,36,71,139.12,2256.370,9729.246,0.232,11985.616
1,012065014,open,2019-09-25 06:56:09,NaN,52.097,80,NaN,NaN,0.0,58.74,5.19765,180.85,40,78,176.74,17301.231,4135.452,4.184,21436.684
2,012065014,open,2019-09-27 06:56:53,NaN,42.169,80,NaN,NaN,0.0,74.34,5.23139,187.05,38,73,268.82,6072.220,4735.696,1.282,10807.915
3,012065014,readiness,2019-10-04 07:04:37,NaN,110.815,74,NaN,NaN,0.0,72.98,4.80254,121.82,71,53,149.81,16425.251,7177.288,2.289,23602.539
4,012065014,readiness,2019-10-07 06:38:42,NaN,91.959,75,8.0,PS,0.0,58.34,4.84465,127.06,60,67,186.36,1712.849,4202.184,0.408,5915.033


### Image

In [7]:
# get the list of items

sourceName = 'umd-athletics'
sourceVariation = '2020-07-07'
videoName = 'capture'

BUCKET = 'video-frame'
cloudPath = f"{sourceName}/{sourceVariation}/{videoName}"
items = list(storageClient.list_blobs(BUCKET, prefix=cloudPath))
items = [t.name for t in items]

print(f"frames: {len(items):,}")

frames: 10,043


In [9]:
# download each image

storageBucket = storageClient.bucket(BUCKET)

for idx, path in enumerate(items):

    # get data from cloud

    blob = storageBucket.blob(path)
    imageTx = blob.download_as_string()
    fileName = os.path.basename(path)

    # save image to disk

    nparr = NP.frombuffer(imageTx, NP.uint8)
    img = CV.imdecode(nparr, CV.IMREAD_COLOR)
    outputPath = os.path.join(MODULE_PATH, 'notebooks', fileName) 
    results = CV.imwrite(outputPath, img)
    print(outputPath)

    # for dev only
    if idx > 2: break

C:\Projects\PraxiService\codebase\common\notebooks\ic-1594136690-01.jpg
C:\Projects\PraxiService\codebase\common\notebooks\ic-1594136690-02.jpg
C:\Projects\PraxiService\codebase\common\notebooks\ic-1594136690-03.jpg
C:\Projects\PraxiService\codebase\common\notebooks\ic-1594136690-04.jpg
C:\Projects\PraxiService\codebase\common\notebooks\ic-1594136690-05.jpg
C:\Projects\PraxiService\codebase\common\notebooks\ic-1594136690-06.jpg
C:\Projects\PraxiService\codebase\common\notebooks\ic-1594136690-07.jpg
